## Board game data compilation

Import libraries

In [ ]:
import pandas as pd
from pandas.io.json import json_normalize
import requests
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import json

Client id for boardgameatlas.com

In [ ]:
client_id = "
client_secret = ""

#### Test qpi query of boardgameatlas.com
Query all, ordering by popularity, receive as a json file, then normalize the json file

In [ ]:
url = "https://www.boardgameatlas.com/api/search?order_by=popularity&ascending=false&skip="+str(0)+"&client_id="
results = requests.get(url).json()

Check results

In [ ]:
results

Convert json to data frame

In [ ]:
games = json_normalize(results['games'])

Look at resulting json normalized file

In [ ]:
games

#### Use a loop to query and build a dataframe by descending popularity
Query by descending popularity, skipping by 100s

In [ ]:
games2 = pd.DataFrame()
for i in range(0,75):
	skip = i*100
	url = "https://www.boardgameatlas.com/api/search?order_by=popularity&ascending=false&skip="+str(skip)+"&client_id="
	results = requests.get(url).json()
	gamedf = json_normalize(results,'games')
	gamepdf = pd.DataFrame(gamedf)
	games2 = games2.append(gamepdf,ignore_index=True, sort=False)

In [ ]:
len(games2)

### Save dataframe to file

In [ ]:
games2.to_csv('Games7400.csv',index=False)

### Read in stored games csv

In [ ]:
games2=pd.read_csv('Games7400.csv')

In [ ]:
games2.columns

##### How many games for children?

In [ ]:
len(gsort[(gsort['min_age']<9)])

In [ ]:
len(gsort[(gsort['min_age']<9)& (gsort['average_user_rating']>3)])

*************

### Get a list of mechanics ids

In [ ]:
# Get a list of the codes for the different types of mechanics
url = 'https://www.boardgameatlas.com/api/game/mechanics?client_id='
results = requests.get(url).json()
mechanics = pd.json_normalize(results['mechanics'])
mechanics

### Exploratory data analysis

In [ ]:
temp = games2[(games2['msrp']<50)&(games2['msrp']>0)]
len(temp)

In [ ]:
temp = temp[(temp['max_players']<10)]
len(temp)

In [ ]:
temp = temp[(temp['max_playtime'] < 150)]

In [ ]:
sns.jointplot(x=temp['msrp'], y=temp['max_playtime'], kind='kde')

Subset games2 data frame by games with different amounts of user ratings

In [ ]:
g100 = games2[(games2['num_user_ratings'] > 99) &  (games2['num_user_ratings'] <200)]

In [ ]:
len(g100)

In [ ]:
g200 = games2[(games2['num_user_ratings'] > 199) &  (games2['num_user_ratings'] <300)]

In [ ]:
len(g200)

In [ ]:
g200id = g200['id']

In [ ]:
g300 = games2[(games2['num_user_ratings'] > 299) &  (games2['num_user_ratings'] <400)]

In [ ]:
len(g300)

In [ ]:
g300id = g300['id']

In [ ]:
g400 = games2[(games2['num_user_ratings'] > 399) &  (games2['num_user_ratings'] <500)]

In [ ]:
len(games2[(games2['num_user_ratings']>499)])

In [ ]:
games2['average_user_rating']
games2.average_user_rating = games2.average_user_rating.astype('float')

In [ ]:
games2.msrp =  games2.msrp.astype('float')

In [ ]:
games2.min_playtime = games2.min_playtime.astype('float')

### Explore prices

In [ ]:
game_sub = games2[(games2.average_user_rating > 0) & (games2.msrp < 151) & (games2.min_playtime <121)]
len(game_sub)

In [ ]:
sns.regplot(x='average_user_rating',y='min_playtime', data=game_sub)

In [ ]:
g0 = games2.msrp[games2.msrp ==0 ]
len(g0)

In [ ]:
# Subset dataframe of just games with no listed MSRP
gFree = games2[games2.msrp ==0 ]
gFree['name'].head() 

In [ ]:
msrpl200 = games2.msrp[(games2.msrp < 200) & (games2.msrp >0)]
len(msrpl200)

In [ ]:
gmsrp200 = games2[(games2.msrp < 200) & (games2.msrp >0)]


In [ ]:
sns.distplot(msrpl100)

In [ ]:
g200names = gmsrp200.id.head()

Query by a game name

In [ ]:
url = "https://www.boardgameatlas.com/api/search?name=Catan&client_id="

Query by reviews for an individual game id

In [ ]:
url_review = "https://www.boardgameatlas.com/api/reviews?game_id=YIBcOZqGkI&client_id="

Query by reviews by an individual reviewer

In [ ]:
url_review_user = "https://www.boardgameatlas.com/api/reviews?username=mordroc&client_id="

#### Get a list of game categories and their tags

In [ ]:
urlcats = 'https://www.boardgameatlas.com/api/game/categories?client_id='
r = requests.get(urlcats).json()
gamecats = json_normalize(r['categories'])
gamecats.describe()

In [ ]:
gamecats.to_csv("gamecats.csv")

In [ ]:
gamecats = pd.read_csv("gamecats.csv")

#### Build a dictionary with game categories as keys and dataframes of boadgame details as values 

In [ ]:
ALLDFS={}
for i in gamecats.index[0:114]:
    qid=gamecats.loc[i,'id']
    url = "https://www.boardgameatlas.com/api/search?categories="+str(qid)+"&client_id="
    results = requests.get(url).json()
    gamedf = json_normalize(results,'games')
    name=gamecats.loc[i,'name']
    #print(name)
    ALLDFS[name]=gamedf

In [ ]:
ALLDFS['Zombies'].name

In [ ]:
import pickle

In [ ]:
# To write a pickle file
with open('ALLDFS2.pickle','wb') as file:
    pickle.dump(ALLDFS2,file)

In [ ]:
# To open a pickle file
with open('ALLDFS.pickle','rb') as file:
    ALLDFS = pickle.load(file)

### Build dataframes summarizing data in different ways by category

In [ ]:
XDF=pd.DataFrame()
for key in ALLDFS.keys():
    x = ALLDFS.get(key)
    XDF.loc[key,'len']=len(x.name)
    XDF.loc[key,'aur']= x['average_user_rating'][(x['average_user_rating'] >0)].mean()
    XDF.loc[key,'minur']= x['average_user_rating'][(x['average_user_rating'] >0)].min() 
    XDF.loc[key,'maxur']= x['average_user_rating'][(x['average_user_rating'] >0)].max()
    XDF.loc[key,'maxplayers']= x['max_players'][(x['max_players'] >0)].mean()
    XDF.loc[key,'minplayers']= x['min_players'][(x['min_players'] >0)].mean()
    XDF.loc[key,'nrat0']= len(x['average_user_rating'][(x['average_user_rating']==0)])
    XDF.loc[key,'nratnz']= len(x['average_user_rating'][(x['average_user_rating']>0)])
    XDF.loc[key,'minptmean']= x['min_playtime'][(x['min_playtime'] >0)].mean()
    XDF.loc[key,'minptmin']= x['min_playtime'][(x['min_playtime'] >0)].min()
    XDF.loc[key,'minptmax']= x['min_playtime'][(x['min_playtime'] >0)].max()
    XDF.loc[key,'maxptmean']= x['max_playtime'][(x['max_playtime'] >0)].mean()
    XDF.loc[key,'maxptmin']= x['max_playtime'][(x['max_playtime'] >0)].min()
    XDF.loc[key,'maxptmax']= x['max_playtime'][(x['max_playtime'] >0)].max()


In [ ]:
XDF=pd.DataFrame()
for key in ALLDFS.keys():
    x = ALLDFS.get(key)
    XDF.loc[key,'len']=len(x.name)
    XDF.loc[key,'nratnz']= len(x['average_user_rating'][(x['average_user_rating']>0)])
    XDF.loc[key,'aur']= x['average_user_rating'][(x['average_user_rating'] >0)].mean()
    XDF.loc[key,'minplayers']= x['min_players'][(x['min_players'] >0)].mean()
    XDF.loc[key,'maxplayers']= x['max_players'][(x['max_players'] >0)].mean()
    XDF.loc[key,'minptmean']= x['min_playtime'][(x['min_playtime'] >0)].mean()
    XDF.loc[key,'maxptmean']= x['max_playtime'][(x['max_playtime'] >0)].mean()

In [ ]:
XDF.head(3)

#### Simple plotting of data distributions amongst categories

In [ ]:
grnz10 = XDF[(XDF.nratnz >10)]

In [ ]:
sns.boxplot(x='aur',data=grnz10)

In [ ]:
sns.boxplot(x='minptmean',data=grnz10)

In [ ]:
sns.boxplot(x='maxptmean',data=grnz10)

In [ ]:
sns.boxplot(x='minplayers',data=grnz10)

In [ ]:
sns.boxplot(x='maxplayers',data=grnz10)

In [ ]:
ALLDFS
XDF2=pd.DataFrame()
for key in ALLDFS2.keys():
    x = ALLDFS2.get(key)
    XDF2.loc[key,'len']=len(x)